In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
import trafficDataUtility as util

In [3]:
path = "C:\\Users\\KangYounKook\\Desktop\\ML_Study\\"

In [4]:
dataset = {}
dataset = util.getDatasetFromFile(path+"sectionInfo.txt")

getDataFromFile  C:\Users\KangYounKook\Desktop\ML_Study\sectionInfo.txt


In [5]:
filename = "PeriodTotalAverage.txt"
util.updateDatasetFromFile(path+filename, dataset)

loadDataFromFile  C:\Users\KangYounKook\Desktop\ML_Study\PeriodTotalAverage.txt
update Dataset [move_count,work_count,ratio]


In [6]:
classes = 10

In [14]:
#filename = "input" + str(classes) +"_all.csv"
filename = "input10_2min.csv"
data = util.loadCsvInput(path,filename)

loadCsvInput  C:\Users\KangYounKook\Desktop\ML_Study\input10_2min.csv


In [20]:
selectedList = dataset.keys()
#selectedList = ['Section697','Section754','Section1212','Section852']

In [33]:
def getFeedXY(Xa,Ya,k,count):
    Yn = Ya[k:count]
    Xn = Xa[:count-k]
    for i in range(1,k):
        Xn = np.c_[Xn, Xa[i:count-k+i]]
    
    m = int((count-k)*0.7)
    Xt = Xn[0:m]
    Xval = Xn[m:count-k]
    Yt = Yn[0:m]
    Yval = Yn[m:count-k]

    return Xt,Xval,Yt,Yval

In [21]:
def get2LayerXY(Xa,Ya):
    Xn = np.c_[Xa[:1438],Xa[1:1439]]
    Yn = Ya[2:1440]
    
    Xt = Xn[0:1000]
    Xval = Xn[1000:1437]
    Yt = Yn[0:1000]
    Yval = Yn[1000:1437]
    return Xt,Xval,Yt,Yval

In [22]:
def get3LayerXY(Xa,Ya):
    Xn = np.c_[Xa[:1437],Xa[1:1438],Xa[2:1439]]
    Yn = Ya[3:1440]
    
    Xt = Xn[0:1000]
    Xval = Xn[1000:1437]
    Yt = Yn[0:1000]
    Yval = Yn[1000:1437]

    return Xt,Xval,Yt,Yval

In [30]:
def get3_2LayerXY(Xa,Ya):
    Xn = np.c_[Xa[:717],Xa[1:718],Xa[2:719]]
    Yn = Ya[3:720]
#    print(Xn.shape)
#    print(Yn.shape)
    Xt = Xn[0:501]
    Xval = Xn[501:711]
    Yt = Yn[0:501]
    Yval = Yn[501:711]
#    print(Xt.shape)
#    print(Xval.shape)
#    print(Yt.shape)
#    print(Yval.shape)

    return Xt,Xval,Yt,Yval    

In [26]:
print(dataset['Section697'])

['B_C', '3804', '3302', ['Section4473', 'Section4141'], ['Section4463'], 2101, 0, 34]


In [29]:
outfilename = "CNN_Result" + str(classes) + ".txt"
f = open(outfilename,'w')

num = 0
for id in selectedList:
    if id not in data.columns:
        continue
    if dataset[id][5] < 1440*2:
        continue
    if dataset[id][7] > 80:
#        print(dataset[id][7])
        continue
        
    Xa = util.getX(id,data,dataset)
    if Xa is None:
#        print('X is none')
        continue
    Ya = np.array(data[id]).reshape(-1,1)
#    print(Ya.shape)

    num += 1
    print(num, id, '-------------------------')
    Xt,Xval,Yt,Yval = get3_2LayerXY(Xa,Ya)
#    Xt,Xval,Yt,Xval = get3LayerXY(Xa,Ya)

    input_size = 15
#    classes = 5
    
    X = tf.placeholder(tf.float32, [None, 15])
    X_img = tf.reshape(X, [-1, 5, 3, 1])
    Y = tf.placeholder(tf.int32, [None, 1])
    Y_one_hot = tf.one_hot(Y, classes)
    Y_one_hot = tf.reshape(Y_one_hot, [-1, classes])
#    print(sess.run(tf.shape(X), feed_dict=feed_dict))
    W1 = tf.Variable(tf.random_normal([2,2,1,10], stddev=0.01))
    L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME')
    L1 = tf.nn.relu(L1)

    W2 = tf.Variable(tf.random_normal([3,3,10,10], stddev=0.01))
    L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='VALID')
    L2 = tf.nn.relu(L2)
    L2 = tf.reshape(L2, [-1, 3*10])

    W3 = tf.Variable(tf.random_normal([30, classes], stddev=0.01))
    #W5 = tf.get_variable("W5", shape=[30, classes], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([classes]))

    hypothesis = tf.matmul(L2,W3) + b3
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y_one_hot))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
    prediction = tf.argmax(hypothesis, 1)
    correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for epoch in range(200):
        c, _ = sess.run([cost,optimizer], feed_dict={X:Xt,Y:Yt})
    
    trainAccuracy = sess.run(accuracy, feed_dict={X:Xt,Y:Yt})
    valAccuracy = sess.run(accuracy, feed_dict={X:Xval,Y:Yval})    
    print('Training:', trainAccuracy)
    print('Validation:', valAccuracy)

    log = id
    log += ',' + dataset[id][0]
    log += ',' + dataset[id][1]
    log += ',' + str(dataset[id][5])
    log += ',' + str(dataset[id][7])
    log += ',' + str(trainAccuracy)
    log += ',' + str(valAccuracy)
    log += '\n'
    f.write(log)
f.close()

1 Section3757 -------------------------
(717, 15)
(717, 1)
(501, 15)
(210, 15)
(501, 1)
(210, 1)
Training: 0.744511
Validation: 0.7
2 Section732 -------------------------
(717, 15)
(717, 1)
(501, 15)
(210, 15)
(501, 1)
(210, 1)
Training: 0.399202
Validation: 0.295238
3 Section3807 -------------------------
(717, 15)
(717, 1)
(501, 15)
(210, 15)
(501, 1)
(210, 1)
Training: 0.53493
Validation: 0.419048
None data[prevId] is None
4 Section760 -------------------------
(717, 15)
(717, 1)
(501, 15)
(210, 15)
(501, 1)
(210, 1)
Training: 0.295409
Validation: 0.333333
5 Section768 -------------------------
(717, 15)
(717, 1)
(501, 15)
(210, 15)
(501, 1)
(210, 1)
Training: 0.309381
Validation: 0.195238
None data[prevId] is None
6 Section1257 -------------------------
(717, 15)
(717, 1)
(501, 15)
(210, 15)
(501, 1)
(210, 1)
Training: 0.600798
Validation: 0.62381
None data[prevId] is None
7 Section1212 -------------------------
(717, 15)
(717, 1)
(501, 15)
(210, 15)
(501, 1)
(210, 1)
Training: 0.6